In [1]:
# 开始训练0-3的mnist模型
import torch 
import torchvision
from torchvision.datasets import mnist # 获取数据集和数据预处理
import matplotlib.pyplot as plt
from torch.utils.data import Subset
from torch.utils.data import DataLoader

data_tf = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize([0.5],[0.5])
    ]
)
data_path = '/home/.faa/data'
batch_size = 256
# 获取数据集
train_data = mnist.MNIST(data_path,train=True,transform=data_tf,download=True)
test_data = mnist.MNIST(data_path,train=False,transform=data_tf,download=True)
train_index = torch.zeros_like(train_data.targets)
test_index = torch.zeros_like(test_data.targets)
for i in range(8):
    train_index |= train_data.targets == i
    test_index |= test_data.targets == i
train_loader_0to7 = DataLoader(Subset(train_data, torch.nonzero(train_index).reshape(-1)), batch_size=batch_size, shuffle=True)
test_loader_0to7 = DataLoader(Subset(test_data, torch.nonzero(test_index).reshape(-1)), batch_size=batch_size, shuffle=True)

In [2]:
train_index = torch.zeros_like(train_data.targets)
test_index = torch.zeros_like(test_data.targets)
for i in range(4):
    train_index |= train_data.targets == i
    test_index |= test_data.targets == i
train_loader_0to3 = DataLoader(Subset(train_data, torch.nonzero(train_index).reshape(-1)), batch_size=batch_size, shuffle=True)
test_loader_0to3 = DataLoader(Subset(test_data, torch.nonzero(test_index).reshape(-1)), batch_size=batch_size, shuffle=True)

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):
    def __init__(self, output_channel, hidden1, hidden2, mlp_hidden):
        super(CNN,self).__init__()
        self.view_input_dim=4*4*hidden2
        self.conv1=nn.Conv2d(1,hidden1,5,1)
        self.conv2=nn.Conv2d(hidden1,hidden2,5,1)
        self.fc1=nn.Linear(self.view_input_dim,mlp_hidden)
        self.fc2=nn.Linear(mlp_hidden,output_channel)

    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=F.max_pool2d(x,2,2)
        x=F.relu(self.conv2(x))
        x=F.max_pool2d(x,2,2)
        x=x.view(-1,self.view_input_dim)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return F.log_softmax(x,dim=1)   

In [26]:
lr=0.005#学习率
momentum=0.5
torch.cuda.set_device(0)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu" )
model=CNN(8, 40, 100, 800).to(device)
optimizer=optim.SGD(model.parameters(),lr=lr,momentum=momentum, weight_decay=0.01)

In [10]:
old = torch.load('0to3/0-3_cnn.ckpt')

In [11]:
for name, g in model.named_parameters():
    print(name, g.shape, old[name].shape)

conv1.weight torch.Size([40, 1, 5, 5]) torch.Size([20, 1, 5, 5])
conv1.bias torch.Size([40]) torch.Size([20])
conv2.weight torch.Size([100, 40, 5, 5]) torch.Size([50, 20, 5, 5])
conv2.bias torch.Size([100]) torch.Size([50])
fc1.weight torch.Size([800, 1600]) torch.Size([500, 800])
fc1.bias torch.Size([800]) torch.Size([500])
fc2.weight torch.Size([8, 800]) torch.Size([4, 500])
fc2.bias torch.Size([8]) torch.Size([4])


In [28]:
import re
for i, (name, g) in enumerate(model.named_parameters()):
    old_g = old[name].data
    opname, wname = name.split('.')
    if re.match('weight', wname):
        if re.match('conv*', opname):
            new_g = g.data
        else: # fc
            new_g = torch.zeros_like(g)
        outd, ind = old_g.shape[0], old_g.shape[1]
        new_g[:outd, :ind] = old_g
    elif re.match('bias', wname):
        new_g = torch.zeros_like(g)
        new_g[:old_g.shape[0]] = old_g
    g.data = new_g

In [29]:
model.eval()
correct=0
num = 0
with torch.no_grad():
    for _,(t_data,t_target) in enumerate(test_loader_0to3):
        t_data,t_target=t_data.to(device),t_target.to(device)
        pred=model(t_data)
        pred_class=pred.argmax(dim=1)
        correct+=pred_class.eq(t_target.view_as(pred_class)).sum().item()
        num += len(t_target)
acc=correct/num
print("accuracy:{}".format(acc))

accuracy:0.9778686552802501


In [30]:
model.eval()
correct=0
num = 0
with torch.no_grad():
    for _,(t_data,t_target) in enumerate(test_loader_0to7):
        t_data,t_target=t_data.to(device),t_target.to(device)
        pred=model(t_data)
        pred_class=pred.argmax(dim=1)
        correct+=pred_class.eq(t_target.view_as(pred_class)).sum().item()
        num += len(t_target)
acc=correct/num
print("accuracy:{}".format(acc))

accuracy:0.5070475240114756


In [31]:
num_epochs=5
losses=[]
acces=[]
step = 0
def loop(model,device,train_loader,test_loader, optimizer):
    global step, losses, acces
    model.train()
    for _,(t_data,t_target) in enumerate(train_loader):
        t_data,t_target=t_data.to(device),t_target.to(device)
        pred=model(t_data)
        loss=F.nll_loss(pred,t_target)
        optimizer.zero_grad()#将上一步的梯度清0
        loss.backward()#重新计算梯度
        optimizer.step()#更新参数
        losses.append(loss.detach())
        step += 1
        if step % 50==0:
            model.eval()
            correct=0
            num = 0
            with torch.no_grad():
                for _,(t_data,t_target) in enumerate(test_loader):
                    t_data,t_target=t_data.to(device),t_target.to(device)
                    pred=model(t_data)
                    pred_class=pred.argmax(dim=1)
                    correct+=pred_class.eq(t_target.view_as(pred_class)).sum().item()
                    num += len(t_target)
            acc=correct/num
            print("step:{},accuracy:{}".format(step, acc))
            acces.append(acc)
            model.train()
    
for epoch in range(num_epochs):
    loop(model,device,train_loader_0to7, test_loader_0to7, optimizer)

step:50,accuracy:0.8318573032306349
step:100,accuracy:0.8934763627292005
step:150,accuracy:0.9170512660596233
step:200,accuracy:0.9297742297617563
step:250,accuracy:0.9391293501309717
step:300,accuracy:0.9439940127229637
step:350,accuracy:0.9503554945740302
step:400,accuracy:0.9549706872895098
step:450,accuracy:0.9577148559311464
step:500,accuracy:0.9592116751902208
step:550,accuracy:0.9619558438318573
step:600,accuracy:0.9637021329674442
step:650,accuracy:0.9651989522265186
step:700,accuracy:0.9663215666708245
step:750,accuracy:0.9673194461768742
step:800,accuracy:0.9669452413621056
step:850,accuracy:0.9691904702507173
step:900,accuracy:0.9694399401272297


In [32]:
torch.save(model.state_dict(), '0-7_cnn.ckpt')
import numpy as np
np.savetxt("losses_0to7_surgery", losses)
np.savetxt("acces_0to7_surgery", acces)

In [ ]:
torch.save(model.state_dict(), '0-7_cnn_surgery.ckpt')